In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from itertools import combinations, groupby
from collections import Counter

In [2]:
orders = pd.read_csv('total_orders.csv')
cluster_data = pd.read_csv('cluster_data')
products = pd.read_csv('products.csv')
aisles = pd.read_csv('aisles.csv')
departments = pd.read_csv('departments.csv')

In [3]:
orders = orders.sort_values(['order_id','product_id'])
orders.head()

,Unnamed: 0,order_id,product_id,add_to_cart_order,reordered,aisle_id,department_id,aisle,department,user_id,cluster
30264547,30264547,1,10246,3,0,83,4,fresh vegetables,produce,112108,5
30264545,30264545,1,11109,2,1,108,16,other creams cheeses,dairy eggs,112108,5
30264548,30264548,1,13176,6,0,24,4,fresh fruits,produce,112108,5
30264544,30264544,1,22035,8,1,21,16,packaged cheese,dairy eggs,112108,5
30264550,30264550,1,43633,5,1,95,15,canned meat seafood,canned goods,112108,5


In [4]:
def freq(iterable):
    if type(iterable) == pd.core.series.Series:
        return iterable.value_counts().rename("freq")
    else: 
        return pd.Series(Counter(iterable)).rename("freq")

    
def order_count(order_item):
    return len(set(order_item.index))


def get_item_pairs(order_item):
    order_item = order_item.reset_index().to_numpy()
    for order_id, order_object in groupby(order_item, lambda x: x[0]):
        item_list = [item[1] for item in order_object]
              
        for item_pair in combinations(item_list, 2):
            yield item_pair
            

def merge_item_stats(item_pairs, item_stats):
    return (item_pairs.merge(item_stats.rename(columns={'freq': 'freqA', 'support': 'supportA'}), left_on='item_A', right_index=True)
                .merge(item_stats.rename(columns={'freq': 'freqB', 'support': 'supportB'}), left_on='item_B', right_index=True))


def merge_item_name(rules, item_name):
    columns = ['itemA','itemB','freqAB','supportAB','freqA','supportA','freqB','supportB', 
               'confidenceAtoB','confidenceBtoA','lift']
    rules = (rules.merge(item_name.rename(columns={'item_name': 'itemA'}), left_on='item_A', right_on='item_id')
                .merge(item_name.rename(columns={'item_name': 'itemB'}), left_on='item_B', right_on='item_id'))

In [5]:
def association_rules(order_item, min_support):

    print("Starting order_item: {:22d}".format(len(order_item)))

    item_stats             = freq(order_item).to_frame("freq")
    item_stats['support']  = item_stats['freq'] / order_count(order_item) 


    qualifying_items       = item_stats[item_stats['support'] >= min_support].index
    order_item             = order_item[order_item.isin(qualifying_items)]

    print("Items with support >= {}: {:15d}".format(min_support, len(qualifying_items)))
    print("Remaining order_item: {:21d}".format(len(order_item)))


    order_size             = freq(order_item.index)
    qualifying_orders      = order_size[order_size >= 2].index
    order_item             = order_item[order_item.index.isin(qualifying_orders)]

    print("Remaining orders with 2+ items: {:11d}".format(len(qualifying_orders)))
    print("Remaining order_item: {:21d}".format(len(order_item)))


    item_stats             = freq(order_item).to_frame("freq")
    item_stats['support']  = item_stats['freq'] / order_count(order_item) 


    item_pair_gen          = get_item_pairs(order_item)


    item_pairs              = freq(item_pair_gen).to_frame("freqAB")
    item_pairs['supportAB'] = item_pairs['freqAB'] / len(qualifying_orders)

    print("Item pairs: {:31d}".format(len(item_pairs)))


    item_pairs              = item_pairs[item_pairs['supportAB'] >= min_support]

    print("Item pairs with support >= {}: {:10d}\n".format(min_support, len(item_pairs)))


    item_pairs = item_pairs.reset_index().rename(columns={'level_0': 'item_A', 'level_1': 'item_B'})
    item_pairs = merge_item_stats(item_pairs, item_stats)
    
    item_pairs['confidenceAtoB'] = item_pairs['supportAB'] / item_pairs['supportA']
    item_pairs['confidenceBtoA'] = item_pairs['supportAB'] / item_pairs['supportB']
    item_pairs['lift']           = item_pairs['supportAB'] / (item_pairs['supportA'] * item_pairs['supportB'])
    
    return item_pairs.sort_values('lift', ascending=False)

In [6]:
cluster_orders = {}
for x in range(0,6):
    i = orders[orders['cluster'] == x]
    i = i.set_index('order_id')['product_id'].rename('item_id')
    cluster_orders[x] = i

In [7]:
group_association_rules_dic = {}
for x in range(0,6):
    group_association_rules_dic[x] = association_rules(cluster_orders[x],.0001)

Starting order_item:                1736771
Items with support >= 0.0001:            6205
Remaining order_item:               1628029
Remaining orders with 2+ items:      224982
Remaining order_item:               1605793
Item pairs:                         1639064
Item pairs with support >= 0.0001:      45261

Starting order_item:                 541123
Items with support >= 0.0001:            1063
Remaining order_item:                538032
Remaining orders with 2+ items:      100195
Remaining order_item:                520562
Item pairs:                          151541
Item pairs with support >= 0.0001:      24497

Starting order_item:                5646738
Items with support >= 0.0001:            7487
Remaining order_item:               5367101
Remaining orders with 2+ items:      512316
Remaining order_item:               5348390
Item pairs:                         5015172
Item pairs with support >= 0.0001:      90722

Starting order_item:               10149979
Items with suppor

In [8]:
for x in group_association_rules_dic:
    group_association_rules_dic[x] = group_association_rules_dic[x].merge(products[['product_id','product_name']]
               .rename(columns = {'product_id':'item_A','product_name':'product_name_A'}), on = 'item_A')
    group_association_rules_dic[x] = group_association_rules_dic[x].merge(products[['product_id','product_name']]
               .rename(columns = {'product_id':'item_B','product_name':'product_name_B'}), on = 'item_B')
    group_association_rules_dic[x] = group_association_rules_dic[x][['item_A','item_B','product_name_A',
                                                                    'product_name_B','freqAB','supportAB','freqA',
                                                                    'supportA','freqB','supportB','confidenceAtoB',
                                                                    'confidenceBtoA','lift']]
    

In [9]:
x = group_association_rules_dic[5]
x.sort_values('lift', ascending = False)

,item_A,item_B,product_name_A,product_name_B,freqAB,supportAB,freqA,supportA,freqB,supportB,confidenceAtoB,confidenceBtoA,lift
0,20153,46949,Eat Your Colors Purples Puree Baby Food,Eat Your Colors Reds Puree Baby Food,132,0.000103,285,0.000223,227,0.000178,0.463158,0.581498,2605.430466
1,29671,38652,Organic Bluephoria Yerba Mate,Yerba Mate Orange Exuberance Tea,160,0.000125,303,0.000237,297,0.000233,0.528053,0.538721,2270.372815
2,6583,38652,Oraganic Lemon Elation Yerba Mate Drink,Yerba Mate Orange Exuberance Tea,131,0.000103,294,0.000230,297,0.000233,0.445578,0.441077,1915.771857
5,6583,29671,Oraganic Lemon Elation Yerba Mate Drink,Organic Bluephoria Yerba Mate,130,0.000102,294,0.000230,303,0.000237,0.442177,0.429043,1863.501156
6,11187,29671,Revel Berry Yerba Mate,Organic Bluephoria Yerba Mate,172,0.000135,423,0.000331,303,0.000237,0.406619,0.567657,1713.648417
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3775,5450,47209,Small Hass Avocado,Organic Hass Avocado,218,0.000171,27849,0.021809,124683,0.097641,0.007828,0.001748,0.080171
15760,4605,22935,Yellow Onions,Organic Yellow Onion,153,0.000120,39382,0.030841,63625,0.049825,0.003885,0.002405,0.077973
13607,5876,47626,Organic Lemon,Large Lemon,175,0.000137,47741,0.037387,80937,0.063383,0.003666,0.002162,0.057833
10960,47209,47766,Organic Hass Avocado,Organic Avocado,523,0.000410,124683,0.097641,99931,0.078257,0.004195,0.005234,0.053601


In [10]:
cluster_aisle_orders = {}
for x in range(0,6):
    i = orders[orders['cluster'] == x]
    i = i.groupby(['order_id','aisle_id'])['product_id'].count().reset_index().set_index('order_id')['aisle_id'].rename('item_id')
    cluster_aisle_orders[x] = i

In [11]:
orders.groupby(['order_id','aisle_id'])['product_id'].count().reset_index().set_index('order_id')['aisle_id'].rename('item_id')


order_id
1           21
1           24
1           83
1           95
1          108
          ... 
3421083     52
3421083     74
3421083     78
3421083     92
3421083    117
Name: item_id, Length: 24329222, dtype: int64

In [12]:
orders[orders['cluster'] == 5]['order_id'].nunique()


1320912

In [13]:
group_aisle_association_rules_dic = {}
for x in range(0,6):
    group_aisle_association_rules_dic[x] = association_rules(cluster_aisle_orders[x],.0001)


Starting order_item:                1181993
Items with support >= 0.0001:             134
Remaining order_item:               1181993
Remaining orders with 2+ items:      219472
Remaining order_item:               1152515
Item pairs:                            8553
Item pairs with support >= 0.0001:       5390

Starting order_item:                 417290
Items with support >= 0.0001:             118
Remaining order_item:                417248
Remaining orders with 2+ items:       97785
Remaining order_item:                397201
Item pairs:                            5724
Item pairs with support >= 0.0001:       3383

Starting order_item:                3697275
Items with support >= 0.0001:             134
Remaining order_item:               3697275
Remaining orders with 2+ items:      512880
Remaining order_item:               3676936
Item pairs:                            8839
Item pairs with support >= 0.0001:       6224

Starting order_item:                7677403
Items with suppor

In [14]:
for x in group_aisle_association_rules_dic:
    group_aisle_association_rules_dic[x] = group_aisle_association_rules_dic[x].merge(aisles[['aisle_id','aisle']]
               .rename(columns = {'aisle_id':'item_A','aisle':'aisle_name_A'}), on = 'item_A')
    group_aisle_association_rules_dic[x] = group_aisle_association_rules_dic[x].merge(aisles[['aisle_id','aisle']]
               .rename(columns = {'aisle_id':'item_B','aisle':'aisle_name_B'}), on = 'item_B')
    group_aisle_association_rules_dic[x] = group_aisle_association_rules_dic[x][['item_A','item_B','aisle_name_A',
                                                                    'aisle_name_B','freqAB','supportAB','freqA',
                                                                    'supportA','freqB','supportB','confidenceAtoB',
                                                                    'confidenceBtoA','lift']]
    group_aisle_association_rules_dic[x] = group_aisle_association_rules_dic[x].rename(columns = {'item_A':'aisle_A','item_B':'aisle_B'})


In [15]:
x = group_aisle_association_rules_dic[5].sort_values('lift',ascending = False)
x.sort_values('lift',ascending = False)

,aisle_A,aisle_B,aisle_name_A,aisle_name_B,freqAB,supportAB,freqA,supportA,freqB,supportB,confidenceAtoB,confidenceBtoA,lift
0,28,62,red wines,white wines,1769,0.001381,7044,0.005498,6509,0.005080,0.251136,0.271778,49.434073
38,124,134,spirits,specialty wines champagnes,361,0.000282,5173,0.004037,2200,0.001717,0.069785,0.164091,40.641921
39,62,134,white wines,specialty wines champagnes,435,0.000340,6509,0.005080,2200,0.001717,0.066831,0.197727,38.921045
40,27,134,beers coolers,specialty wines champagnes,369,0.000288,5565,0.004343,2200,0.001717,0.066307,0.167727,38.616304
37,28,134,red wines,specialty wines champagnes,424,0.000331,7044,0.005498,2200,0.001717,0.060193,0.192727,35.055488
...,...,...,...,...,...,...,...,...,...,...,...,...,...
150,120,124,yogurt,spirits,683,0.000533,435847,0.340175,5173,0.004037,0.001567,0.132032,0.388129
146,91,124,soy lactosefree,spirits,402,0.000314,273737,0.213649,5173,0.004037,0.001469,0.077711,0.363733
5165,27,92,beers coolers,baby food formula,142,0.000111,5565,0.004343,90377,0.070538,0.025517,0.001571,0.361741
5164,28,92,red wines,baby food formula,159,0.000124,7044,0.005498,90377,0.070538,0.022572,0.001759,0.320002


In [16]:
cluster_data['cluster'].value_counts() / cluster_data['cluster'].value_counts().sum()


3    0.356687
5    0.352152
2    0.159406
0    0.074667
1    0.038539
4    0.018549
Name: cluster, dtype: float64

In [17]:
for x in group_association_rules_dic:
    group_association_rules_dic[x].to_csv('group'+str(x)+'_item_rules.csv')


In [18]:
for x in group_aisle_association_rules_dic:
    group_aisle_association_rules_dic[x].to_csv('group'+str(x)+'_aisle_rules.csv')
